## Semester 1 Project Submission

Please fill out:
* Student name: 
* Instructor name: 


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

# Dylan Orndorf-Ronk

# Alex Bonczkiewicz

In [4]:
import sqlite3
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

conn = sqlite3.connect('zippedData/im.db')

In [7]:
#test

DatabaseError: Execution failed on sql '
SELECT *
FROM principals;
': no such table: principals

# Alberto Torres

# Travis McCloughan

# Jaemin Lee